## Dependencies

In [1]:
import glob
import warnings
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate, Add, Average
from googleqa_utilityscript import *
from transformers import BertConfig, BertTokenizer, TFBertModel


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/bert-base-uncased-huggingface-transformer/bert-base-uncased-tf_model.h5'
VOCAB_PATH = '/kaggle/input/bert-base-uncased-huggingface-transformer/bert-base-uncased-vocab.txt'
model_path_list = glob.glob('/kaggle/input/146-googleq-a-train-3fold-bert-base-unc-avg/' + '*1.h5')
model_path_list += glob.glob('/kaggle/input/146-googleq-a-train-3fold-bert-base-unc-avg/' + '*2.h5')
model_path_list += glob.glob('/kaggle/input/146-googleq-a-train-3fold-bert-base-unc-avg/' + '*3.h5')
model_path_list.sort()
print('Models to predict:')
print(*model_path_list, sep = "\n")

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict:
/kaggle/input/146-googleq-a-train-3fold-bert-base-unc-avg/model_fold_1.h5
/kaggle/input/146-googleq-a-train-3fold-bert-base-unc-avg/model_fold_2.h5
/kaggle/input/146-googleq-a-train-3fold-bert-base-unc-avg/model_fold_3.h5
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = BertTokenizer.from_pretrained(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
bert_config = BertConfig()
bert_config.output_hidden_states=True

def model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_model = TFBertModel.from_pretrained(BERT_PATH, config=bert_config)
    sequence_output, pooled_output, hidden_states = bert_model([input_word_ids, input_masks, segment_ids])
    
    h12 = tf.reshape(hidden_states[-1][:, 0],(-1, 1, 768))
    h11 = tf.reshape(hidden_states[-2][:, 0],(-1, 1, 768))
    h10 = tf.reshape(hidden_states[-3][:, 0],(-1, 1, 768))
    h09 = tf.reshape(hidden_states[-4][:, 0],(-1, 1, 768))
    
    x = Average()([h12,h11, h10, h09])

    x = GlobalAveragePooling1D()(x)
    x = Dropout(0.2)(x)
    output = Dense(N_CLASS, activation="sigmoid", name="output")(x)

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)
    
    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test) / len(model_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.949900,0.713147,0.238588,0.479536,0.430988,0.436307,0.699098,0.719989,0.595070,...,0.924349,0.915328,0.601514,0.978003,0.956171,0.825978,0.026208,0.035221,0.911244,0.907501
1,46,0.892549,0.574181,0.006262,0.767736,0.794494,0.933851,0.567344,0.477546,0.051978,...,0.730966,0.950001,0.657597,0.978610,0.981679,0.906695,0.921337,0.102934,0.069823,0.892257
2,70,0.906173,0.672178,0.020527,0.782555,0.886769,0.950308,0.607424,0.481488,0.113604,...,0.852359,0.949332,0.595647,0.977537,0.972748,0.875104,0.092708,0.047454,0.906404,0.905298
3,132,0.890750,0.457822,0.008480,0.731876,0.731359,0.937836,0.539876,0.450380,0.051666,...,0.673257,0.959220,0.679860,0.976856,0.983855,0.908715,0.875495,0.135051,0.521634,0.891326
4,200,0.944369,0.451543,0.033290,0.866632,0.674426,0.867786,0.675701,0.584797,0.061418,...,0.700479,0.902536,0.616539,0.974066,0.965643,0.829029,0.307216,0.082170,0.677985,0.894890


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.899985,0.626405,0.039995,0.733839,0.814339,0.856993,0.587299,0.505782,0.210438,...,0.814148,0.945286,0.659078,0.972200,0.974396,0.880143,0.539939,0.117771,0.503986,0.912506
std,2812.670060,0.041464,0.130759,0.081367,0.107157,0.110427,0.133425,0.050382,0.091319,0.204289,...,0.081996,0.021861,0.045588,0.009038,0.011909,0.039814,0.342429,0.057315,0.300383,0.020733
min,39.000000,0.765665,0.357010,0.003708,0.205831,0.186146,0.226934,0.451239,0.345422,0.006591,...,0.625184,0.822920,0.475780,0.930883,0.910731,0.701767,0.003705,0.009034,0.019170,0.786538
25%,2572.000000,0.873843,0.510998,0.006999,0.677048,0.767609,0.842253,0.551624,0.442258,0.057665,...,0.745061,0.938754,0.628614,0.968394,0.970097,0.863783,0.186364,0.079354,0.226979,0.902806
50%,5093.000000,0.902593,0.621956,0.010872,0.739891,0.828294,0.907191,0.576662,0.476991,0.120722,...,0.817384,0.951252,0.662896,0.974065,0.977398,0.889604,0.637284,0.120082,0.510690,0.914076
75%,7482.000000,0.930456,0.733613,0.027545,0.804997,0.884749,0.936482,0.616092,0.547408,0.299003,...,0.883700,0.958870,0.692121,0.977853,0.982538,0.905685,0.867354,0.153122,0.778626,0.925438
max,9640.000000,0.977889,0.896968,0.728257,0.965879,0.983533,0.977523,0.744098,0.790491,0.891213,...,0.959046,0.982545,0.776138,0.988092,0.993170,0.969733,0.970868,0.325365,0.988705,0.957645
